In [1]:
from trainer import Trainer, TrainerConfig
from mingpt_utils import set_seed
from model import GPT, GPTConfig
import torch
from utils import *

from torch.utils.tensorboard import SummaryWriter
from mingpt_utils import sample
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Available devices: ", torch.cuda.device_count())
print("torch version:", torch.__version__)
print("cudnn version:", torch.backends.cudnn.version())
print("cuda version:", torch.version.cuda)

Available devices:  2
torch version: 2.2.0a0+81ea7a4
cudnn version: 8907
cuda version: 12.3


In [3]:
max_length = 1024
id = 0
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

In [5]:
pad_tokens = 0
elements = 0
for song in train:
    for e in song:
        if e == '<pad>':
            pad_tokens += 1
        elements += 1

percentage = pad_tokens / elements * 100
print(f"Pad tokens: {pad_tokens}, Percentage: {percentage:.2f}%") 

Pad tokens: 16773876, Percentage: 37.86%


In [5]:
#Convert midi into dtype int
midi_train = midi_train.astype(int)
midi_test = midi_test.astype(int)

In [6]:
print(train.shape, test.shape, midi_train.shape, midi_test.shape)

dataset = TokenDatasetMidi(train, midi_train,  max_length, tokens)
validation = TokenDatasetMidi(test, midi_test, max_length, tokens)

(43272, 1024) (4800, 1024) (43272, 1024, 8) (4800, 1024, 8)
data has 43272 pieces, 198 unique tokens.
data has 4800 pieces, 198 unique tokens.


In [7]:
stoi = { tk:i for i,tk in enumerate(tokens) }
itos = { i:tk for i,tk in enumerate(tokens) }

print(stoi)

{'.': 0, '/': 1, '0.3997395833333333': 2, '0.4440104166666667': 3, '0.5': 4, '0.5703125': 5, '0.6666666666666666': 6, '0.75': 7, '0.7994791666666666': 8, '0.8880208333333334': 9, '1.0': 10, '1.1419270833333333': 11, '1.3333333333333333': 12, '1.5': 13, '1.5989583333333333': 14, '1.7135416666666667': 15, '128 Feel': 16, '2.0': 17, '2.25': 18, '2.3997395833333335': 19, '2.6666666666666665': 20, '3.0': 21, '4.0': 22, ':|': 23, '<end>': 24, '<pad>': 25, '<start>': 26, '<style>': 27, 'A': 28, 'A major': 29, 'A minor': 30, 'A#': 31, 'A##': 32, 'Ab': 33, 'Ab major': 34, 'Ab minor': 35, 'Abb': 36, 'Afoxé': 37, 'Afro': 38, 'B': 39, 'B major': 40, 'B minor': 41, 'B#': 42, 'B##': 43, 'Baião': 44, 'Ballad': 45, 'Bb': 46, 'Bb major': 47, 'Bb minor': 48, 'Bbb': 49, 'Blues': 50, 'Bolero': 51, 'Bolero-Cha': 52, 'Bossa': 53, 'C': 54, 'C major': 55, 'C minor': 56, 'C#': 57, 'C##': 58, 'Calypso': 59, 'Cb': 60, 'Cbb': 61, 'Cha Cha': 62, 'Chacarera': 63, 'Choro': 64, 'Country Ballad': 65, 'D': 66, 'D major

In [8]:
import wandb
#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="music_gpt_new_voicing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 3e-5,
    "architecture": "Transformer - minGPT",
    "dataset": "chords from iRealPro",
    "epochs": 90,
    }
)

03/17/2024 21:27:03 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dazzid (music_gpt). Use `wandb login --relogin` to force relogin


In [9]:
# import subprocess

# # Example command to list processes using GPU (this won't run here due to sandbox restrictions)
# output = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
# print(output.stdout)

In [10]:
epochs = 270
embedding = 256
heads = 4
layers = 4
batch_size = 128
learning_rate = 3e-5
num_workers = 4
midi_vocab = 128
token_size = len(tokens)

mconf = GPTConfig(len(tokens), dataset.block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)
MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"
print(MODEL_NAME)

session_model = load_model(MODEL_NAME, session_model)

if (session_model == None):
    #mconf = GPTConfig(len(tokens), dataset.block_size, n_layer=layers, n_head=heads, n_embd=embbedings)
    session_model = GPT(mconf)
    tconf = TrainerConfig(max_epochs=epochs, 
                          batch_size=batch_size, 
                          learning_rate=learning_rate, 
                          num_workers=num_workers
                          )
    writer = SummaryWriter(log_dir='../runs/'+'logs') 
    trainer = Trainer(session_model, dataset, validation, tconf, writer)
    trainer.train()
    save_model(MODEL_NAME, session_model)
    # [optional] finish the wandb run, necessary in notebooks
    wandb.finish()

03/17/2024 21:27:06 - INFO - model -   number of parameters: 3.269632e+06
03/17/2024 21:27:06 - INFO - model -   number of parameters: 3.269632e+06


../models/model_epochs->120_heads->4_embd->256_batch->128_new_midi_embeddings


epoch 1 iter 338: train loss 3.09026. lr 3.000000e-05: 100%|██████████| 339/339 [01:43<00:00,  3.28it/s]
03/17/2024 21:28:51 - INFO - trainer -   epoch train loss: 3.514809


train loss: 3.514809305337326


03/17/2024 21:28:56 - INFO - trainer -   test loss: 3.023743
epoch 2 iter 338: train loss 2.17359. lr 3.000000e-05: 100%|██████████| 339/339 [01:42<00:00,  3.32it/s]
03/17/2024 21:30:39 - INFO - trainer -   epoch train loss: 2.573770


...

train loss: 0.227778125521356


03/18/2024 01:02:10 - INFO - trainer -   test loss: 0.227884


Model ../models/model_epochs->120_heads->4_embd->256_batch->128_new_midi_embeddings saved


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train loss,█▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation loss,█▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
hits@1,▁▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇████████████████████
hits@3,▁▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇████████████████████████
hits@5,▁▅▆▆▆▇▇▇▇▇▇▇▇▇██████████████████████████
Train loss,0.22778
Validation loss,0.22788
hits@1,0.92924
hits@3,0.98254
hits@5,0.99043
